# Analysis of nitroxides radicals using molecular descriptors and PCA #

In [147]:
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import scipy
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from ipywidgets import interact

## Total avalaible descriptors, 2&3D ##

In [148]:
n_all = len(Calculator(descriptors, ignore_3D=False).descriptors)
n_2D = len(Calculator(descriptors, ignore_3D=True).descriptors)
print("2D:    {:5}\n3D:    {:5}\n------------\ntotal: {:5}".format(n_2D, n_all - n_2D, n_all))

2D:     1613
3D:      213
------------
total:  1826


In [149]:
# Set-up descriptors calcualators

calc = Calculator(descriptors, ignore_3D=True)
ABCI = Calculator(descriptors.ABCIndex)
AB = Calculator(descriptors.AcidBase)
Amatrix = Calculator(descriptors.AdjacencyMatrix)
IC = Calculator(descriptors.InformationContent)
Wiener = Calculator(descriptors.WienerIndex)
Weight = Calculator(descriptors.Weight)
SlogP = Calculator(descriptors.SLogP)
TopoPSA = Calculator(descriptors.TopoPSA)

## Import Dataset ##

In [150]:
nox = pd.read_csv('nitroxides.csv', sep=';')
groups = ['piperidin','pyrrolidin','indolin']
nox['group']='others'
nox['g_code']=0
for i in range(len(groups)):
    for j in range(len(nox)):
        if groups[i] in str(nox.at[j,'IUPAC']):
            nox.at[j,'group']=groups[i]
            nox.at[j,'g_code']=i+1
nox

,code,IUPAC,name,SMILES,group,g_code
0,A,"N,N-ditert-butyl nitroxide",DBN,CC(C)(C)N([O])C(C)(C)C,others,0
1,B,"2,2,6,6-tetramethyl-piperidin-1-yloxyl",TEMPO,CC1(C)CCCC(C)(C)N1[O],piperidin,1
2,C,"4-amino-2,2,6,6-tetramethyl-piperidin-1-yloxyl",4-amino-TEMPO,CC1(C)CC(N)CC(C)(C)N1[O],piperidin,1
3,D,"4-hydroxy-2,2,6,6-tetramethyl-piperidin-1-yloxyl",4-hydroxy-TEMPO,CC1(C)CC(O)CC(C)(C)N1[O],piperidin,1
4,E,"3-carboxy-2,2,6,6-tetramethyl-piperidin-1-yloxyl",3-carboxy-TEMPO,CC1(C)CCC(C(O)=O)C(C)(C)N1[O],piperidin,1
5,F,"3,4-dicarboxy-2,2,6,6-tetramethyl-piperidin-1-...","3,4-dicarboxy-TEMPO",CC1(C)CC(C(C(O)=O)C(C)(C)N1[O])C(O)=O,piperidin,1
6,G,"3,5-dicarboxy-2,2,6,6-tetramethyl-piperidin-1-...","3,5-dicarboxy-TEMPO",CC1(C)C(CC(C(O)=O)C(C)(C)N1[O])C(O)=O,piperidin,1
7,H,"4-carboxy-2,2,6,6-tetraethylpiperidin-1-yloxyl",4-carboxy-TEEPO,CCC1(CC)CC(CC(CC)(CC)N1[O])C(O)=O,piperidin,1
8,I,"2,2,5,5-tetramethyl-pyrrolidin-1-yloxyl",PROXYL,CC1(C)CCC(C)(C)N1[O],pyrrolidin,2
9,J,"3-amino-2,2,5,5-tetramethyl-pyrrolidin-1-yloxyl",3-amino-PROXYL,CC1(C)CC(N)C(C)(C)N1[O],pyrrolidin,2


## Compute chosen molecular descriptors ##

In [151]:
nox_mol=[Chem.MolFromSmiles(smi) for smi in nox['SMILES']]

In [152]:
nox_ABCI = ABCI.pandas(nox_mol)
module=[]
for i in range(len(nox_ABCI.T)):
    module.append('ABC_Index')
nox_ABCI.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_ABCI.columns)], 
                                             names=['module', 'descriptor'])
nox_AB = AB.pandas(nox_mol)
module=[]
for i in range(len(nox_AB.T)):
    module.append('Acid_Base')
nox_AB.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_AB.columns)], 
                                             names=['module', 'descriptor'])
nox_AM = Amatrix.pandas(nox_mol)
module=[]
for i in range(len(nox_AM.T)):
    module.append('Adjacency_Matrix')

nox_AM.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_AM.columns)], 
                                             names=['module', 'descriptor'])
nox_IC = IC.pandas(nox_mol)
module=[]
for i in range(len(nox_IC.T)):
    module.append('Information_Content')
nox_IC.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_IC.columns)], 
                                             names=['module', 'descriptor'])
nox_Wiener = Wiener.pandas(nox_mol)
module=[]
for i in range(len(nox_Wiener.T)):
    module.append('Wiener_Index')
nox_Wiener.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_Wiener.columns)], 
                                             names=['module', 'descriptor'])
nox_Weight = Weight.pandas(nox_mol)
module=[]
for i in range(len(nox_Weight.T)):
    module.append('Weight')
nox_Weight.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_Weight.columns)], 
                                             names=['module', 'descriptor'])
nox_SlogP = SlogP.pandas(nox_mol)
module=[]
for i in range(len(nox_SlogP.T)):
    module.append('SLogP')
nox_SlogP.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_SlogP.columns)], 
                                             names=['module', 'descriptor'])
nox_TopoPSA = TopoPSA.pandas(nox_mol)
module=[]
for i in range(len(nox_TopoPSA.T)):
    module.append('TopoPSA')
nox_TopoPSA.columns = pd.MultiIndex.from_arrays([np.array(module),np.array(nox_TopoPSA.columns)], 
                                             names=['module', 'descriptor'])

100%|██████████| 23/23 [00:00<00:00, 610.83it/s]


In [153]:
#nox_ds = pd.concat([nox_ABCI,nox_AB,nox_AM,nox_IC,nox_Wiener,nox_Weight,nox_SlogP,nox_TopoPSA], axis=1)
nox_ds = pd.concat([nox_ABCI,nox_AB,nox_IC,nox_Wiener,nox_Weight,nox_SlogP,nox_TopoPSA], axis=1)
nox_ds.index = nox['name']
#nox_ds.count()

In [154]:
nox_ds.head()

module           ABC_Index            Acid_Base       Information_Content  \
descriptor             ABC      ABCGG     nAcid nBase                 IC0   
name                                                                        
DBN               7.303643   7.754364         0     0            1.269546   
TEMPO             8.400020   8.159052         0     0            1.285982   
4-amino-TEMPO     9.216516   8.946995         0     1            1.365811   
4-hydroxy-TEMPO   9.216516   8.946995         0     0            1.387291   
3-carboxy-TEMPO  10.638070  10.361201         1     0            1.467724   

module                                                             ...  \
descriptor            IC1       IC2       IC3       IC4       IC5  ...   
name                                                               ...   
DBN              1.501340  1.501340  1.501340  1.501340  1.501340  ...   
TEMPO            1.760962  2.425935  2.615928  2.615928  2.615928  ...   
4-amino-TEMPO    2.261072  2.856791  2.856791  2.856791  2.856791  ...   
4-hydroxy-TEMPO  2.190662  2.806239  2.806239  2.806239  2.806239  ...   
3-carboxy-TEMPO  2.453535  3.155639  3.405639  3.780639  3.780639  ...   

module                                Wiener_Index           Weight            \
descriptor           ZMIC4      ZMIC5        WPath WPol          MW       AMW   
name                                                                            
DBN              30.358961  30.358961          111   12  144.138839  5.147816   
TEMPO            27.794522  27.794522          134   17  156.138839  5.384098   
4-amino-TEMPO    28.300953  28.300953          170   19  171.149738  5.520959   
4-hydroxy-TEMPO  28.519125  28.519125          170   19  172.133754  5.737792   
3-carboxy-TEMPO  23.215084  23.215084          265   25  200.128668  6.254021   

module            SLogP              TopoPSA          
descriptor        SLogP      SMR TopoPSA(NO) TopoPSA  
name                                                  
DBN              2.2309  42.0315       23.14   23.14  
TEMPO            2.3750  44.5345       23.14   23.14  
4-amino-TEMPO    1.3122  47.8929       49.16   49.16  
4-hydroxy-TEMPO  1.3458  45.9243       43.37   43.37  
3-carboxy-TEMPO  1.6857  51.0433       60.44   60.44  

[5 rows x 54 columns]

## PCA ##

In [155]:
# PCA
x = nox_ds.copy()
pca = PCA()
xs = scale(x)
x_red = pca.fit_transform(xs)
V = pca.components_
nPC = np.arange(len(V))+1
dfV = pd.DataFrame(V, columns=[x.columns.levels[1]], index=[nPC])
EV_cum = pd.DataFrame(np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100),
                  columns =['% Explained Variance'], index=[nPC])

In [156]:
print ('PCs = ', len(V))

PCs =  23


In [157]:
EV_cum.head(10).T

,1,2,3,4,5,6,7,8,9,10
% Explained Variance,49.91,74.84,88.79,93.37,96.24,97.88,98.85,99.55,99.71,99.81


### PRESS plot ##

In [158]:
Q=100*pca.explained_variance_ratio_
px.scatter(Q, x= nPC, y= Q)

In [159]:
# Calculate ellipse bounds and plot with scores
theta = np.concatenate((np.linspace(-np.pi, np.pi, 50), np.linspace(np.pi, -np.pi, 50)))
circle = np.array((np.cos(theta), np.sin(theta)))
sigma = np.cov(np.array((x_red[:, 0], x_red[:, 1])))
ed = np.sqrt(scipy.stats.chi2.ppf(0.95, 2))
ell = np.transpose(circle).dot(np.linalg.cholesky(sigma) * ed)
a, b = np.max(ell[: ,0]), np.max(ell[: ,1]) #95% ellipse bounds
t = np.linspace(0, 2 * np.pi, 100)

### SCORE plot ###

In [160]:
# Score plot
score=pd.DataFrame(x_red)
index = np.arange(1,len(V)+1)
index = index.astype('int')
index = index.astype('str')
score.columns=index
@interact(PCa=index[:10], PCb=index[:10])
def sel(PCa, PCb):
    ev1 = int(PCa)-1
    ev2 = int(PCb)-1
    xtit = 'PC'+PCa+' - explained variance = '+str(
        np.round(100*pca.explained_variance_ratio_[ev1],decimals=2))+'%'
    ytit = 'PC'+PCb+' - explained variance = '+str(np.round(100*pca.explained_variance_ratio_[ev2],decimals=2))+'%'
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=score[PCa], y=score[PCb], mode='markers', text=x.index,
                             marker=dict(symbol=[200], line_width=2, size=10, color=nox.g_code)))
    fig.add_trace(go.Scatter(x=[score.loc[22,PCa]], y=[score.loc[22,PCb]], mode='markers', text='TEDIO',
                             marker=dict(symbol=[17], line_width=1, size=15, color='lime')))
    fig.add_trace(go.Scatter(x=a * np.cos(t),y=b * np.sin(t),mode='lines',
                        line=dict(color='lightgreen', width=2, dash='dash')))
    fig.update_layout(height=600, width=800, title='PCA - Score plot', 
                  xaxis_title=xtit,yaxis_title=ytit, showlegend=False, xaxis_zeroline=True, yaxis_zeroline=True, 
                  xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')
    return fig.show()

interactive(children=(Dropdown(description='PCa', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'),…

In [161]:
@interact(PCa=index[:10], PCb=index[:10])
def sel(PCa, PCb):
    return px.scatter(score, x=score[PCa], y=score[PCb], color=nox.group, hover_name=nox_ds.index)

interactive(children=(Dropdown(description='PCa', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'),…

In [162]:
x1d = nox_ds.copy()
x1d.columns = nox_ds.columns.levels[1]
@interact(var= x1d.columns)
def f(var):
    return px.scatter(x1d, x=x1d.index, y=var, hover_name=x1d.index, color=nox.group)

interactive(children=(Dropdown(description='var', options=('ABC', 'ABCGG', 'AMW', 'BIC0', 'BIC1', 'BIC2', 'BIC…

### LOADINGS plot ###

In [163]:
Loadings = dfV.T.copy()
Loadings.index = nox_ds.columns
Loadings.columns=np.arange(len(dfV))
D=np.array(list(Loadings.index))
Loadings['descriptors']=D[:,1]
Loadings['module']=D[:,0]
Loadings['mod_code']=0
load_arr = np.array(Loadings.copy())
load=pd.DataFrame(load_arr)
load.columns=Loadings.columns

In [164]:
module = list(Loadings.index.levels[0])
for i in range(len(module)):
    for j in range(len(Loadings)):
        if module[i] == str(load.at[j,'module']):
            load.at[j,'mod_code']=i+1

In [168]:
@interact(PCa=1+np.arange(10),PCb=1+np.arange(10))
def sel(PCa, PCb):
    ev1 = PCa-1
    ev2 = PCb-1
    xtit = 'PC'+str(PCa)+' - explained variance = '+str(
        np.round(100*pca.explained_variance_ratio_[ev1],decimals=2))+'%'        
    ytit = 'PC'+str(PCb)+' - explained variance = '+str(
        np.round(100*pca.explained_variance_ratio_[ev2],decimals=2))+'%'
    ff=go.Figure()
    ff.add_trace(go.Scatter(x=load[PCa],y=load[PCb],mode='markers+text',
                           marker=dict(symbol=[200],
                                       color=load['mod_code'], colorscale='rainbow',
                                       line_width=2, size=10),
                           text=load['descriptors'], textposition='top left', hovertext=load['module']))
    ff.update_layout(height=800, width=1000, title='PCA - Loadings plot', xaxis_title=xtit,yaxis_title=ytit, 
                xaxis_zeroline=True, yaxis_zeroline=True, xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')
    return ff.show()

interactive(children=(Dropdown(description='PCa', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1), Dropdown(…